
- Issue 1  
  How to create my own sdf file for testing purposes  
  https://github.com/angeladai/sgnn/issues/1  
  https://github.com/angeladai/sgnn/tree/master/datagen/GenerateScans

## Import

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt

import hydra
from hydra.experimental import compose, initialize
from omegaconf import OmegaConf



# autoreload python modules on the fly when its source is changed
%load_ext autoreload
%autoreload 2

In [2]:
# sources dir
DIR_SGNN = Path('.').absolute().parent.parent / 'sgnn'
if str(DIR_SGNN) not in sys.path:
    sys.path.append(str(DIR_SGNN))

In [3]:
from model_lightning import LightningTemplateModel
from utils import dict_flatten, watermark, log_text_as_artifact
import scene_dataloader
import data_util

from log import log

## Config

In [4]:
config_path = "../../sgnn/configs"
config_name = "example_01.yaml"

assert Path(config_path).exists()

In [5]:
initialize(config_path=config_path, job_name="test_app")
cfg = compose(config_name="example_01.yaml")

In [6]:
#print(OmegaConf.to_yaml(cfg))
cfg

{'save': './logs', 'model': {'input_dim': '128-64-64', 'encoder_dim': 8, 'coarse_feat_dim': 16, 'refine_feat_dim': 16, 'no_pass_occ': False, 'no_pass_feats': False, 'use_skip_sparse': 1, 'use_skip_dense': 1, 'logweight_target_sdf': True}, 'train': {'retrain': '', 'num_hierarchy_levels': 4, 'num_iters_per_level': 2000, 'num_overfit_val': 160, 'truncation': 3.0, 'batch_size': 8, 'num_workers_train': 4, 'num_workers_valid': 4, 'start_epoch': 0, 'max_epochs': 5, 'save_epoch': 1, 'lr': 0.001, 'decay_lr': 10, 'weight_decay': 0.0, 'weight_sdf_loss': 1.0, 'weight_missing_geo': 5.0, 'vis_dfs': 0, 'use_loss_masking': False, 'no_loss_masking': True, 'scheduler_step_size': 0, 'input_nf': 1}, 'PL': {'gpus': 1, 'distributed_backend': 'dp', 'precision': 32, 'logger': True, 'checkpoint_callback': True, 'early_stop_callback': False, 'gradient_clip_val': 0, 'overfit_batches': 0.0, 'accumulate_grad_batches': 1, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0, 'limit_test_batches': 1.0, 'val_check_in

# Data

In [7]:
root_dir = DIR_SGNN
data_path = root_dir / cfg.data.data_path
train_file_list = root_dir / cfg.data.train_file_list
val_file_list = root_dir / cfg.data.val_file_list


In [8]:
train_files, val_files = data_util.get_train_files(data_path, train_file_list, val_file_list)

In [9]:
#train_files

## By PL model

In [10]:
pl_model = LightningTemplateModel(cfg)

GenModel:
  encoder_dim 8
  input_dim (128, 64, 64)
  input_nf 1
  nf_coarse 16
  nf 16
  num_hierarchy_levels 4
  pass_occ True
  pass_feats True
  use_skip_sparse 1
  use_skip_dense 1
  truncation 3
[TSDFEncoder] params encode sparse 37048
[TSDFEncoder] params encode dense 99552
[TSDFEncoder] params decode dense 232040
#params encoder 344032
#params refinement 242502
#params surfacepred 57201


In [11]:
pl_model.files_root_dir = root_dir
pl_model.files_root_dir

PosixPath('/home/goodok/p_ssd/sgnn/sgnn')

In [12]:
train_dl = pl_model.train_dataloader()

Training data loader called.


#train files =  85564
#val files =  3095


In [13]:
valid_dl = pl_model.val_dataloader()

Validation data loader called.


val_dataset 3095


In [14]:
item = train_dl.dataset[0]
item

{'name': '17DRP5sb8fy_room0__0',
 'input': [tensor([[  0,  44,  25],
          [  0,  44,  26],
          [  0,  45,  25],
          ...,
          [127,  61,  63],
          [127,  62,  63],
          [127,  63,  63]]),
  tensor([[-1.3847],
          [-1.4611],
          [-0.7213],
          ...,
          [-2.9934],
          [-2.8205],
          [-2.8752]])],
 'sdf': tensor([[[[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           ...,
           [   -inf,    -inf,    -inf,  ..., -4.1587, -4.1637, -4.1681],
           [   -inf,    -inf,    -inf,  ..., -4.1358, -4.1286, -4.1257],
           [   -inf,    -inf,    -inf,  ..., -4.1205, -4.1300, -4.1340]],
 
          [[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -

In [15]:
item2 = train_dl.dataset[0]
item2

{'name': '17DRP5sb8fy_room0__0',
 'input': [tensor([[  0,  44,  25],
          [  0,  44,  26],
          [  0,  45,  25],
          ...,
          [127,  61,  63],
          [127,  62,  63],
          [127,  63,  63]]),
  tensor([[-1.3847],
          [-1.4611],
          [-0.7213],
          ...,
          [-2.9934],
          [-2.8205],
          [-2.8752]])],
 'sdf': tensor([[[[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           ...,
           [   -inf,    -inf,    -inf,  ..., -4.1587, -4.1637, -4.1681],
           [   -inf,    -inf,    -inf,  ..., -4.1358, -4.1286, -4.1257],
           [   -inf,    -inf,    -inf,  ..., -4.1205, -4.1300, -4.1340]],
 
          [[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -

In [16]:
log(item2)

   name          '17DRP5sb8fy_room0__0'
   input         [tensor([[  0,  44,  25],
        [  0,  44,  26],
        [  0,  45,  25],
        ...,
        [127,  61,  63],
        [127,  62,  63],
        [127,  63,  63]]), tensor([[-1.3847],
        [-1.4611],
        [-0.7213],
        ...,
        [-2.9934],
        [-2.8205],
        [-2.8752]])]
   sdf           shape: (1, 128, 64, 64)  dtype: torch.float32  min:       -inf,  max:    4.99989,  mean:       -inf
   world2grid    shape: (4, 4)          dtype: torch.float32  min:  -16.14397,  max:  564.01160,  mean:   43.93557
   known         shape: (1, 128, 64, 64)  dtype: torch.uint8    min:    0.00000,  max:    7.00000,  mean:    1.32283
   hierarchy     [tensor([[[[    -inf,     -inf,     -inf,  ..., -20.0651, -14.5297, -11.6725],
          [    -inf,     -inf,     -inf,  ..., -16.7984,  -9.7017,  -5.3165],
          [    -inf,     -inf,     -inf,  ...,     -inf,     -inf,  -4.0711],
          ...,
          [    -inf,  -4.9292,  

In [17]:
log('h0', item2['hierarchy'][0])
log('h1', item2['hierarchy'][1])
log('h2', item2['hierarchy'][2])

h0            shape: (1, 16, 8, 8)   dtype: torch.float32  min:       -inf,  max:   21.19447,  mean:       -inf
h1            shape: (1, 32, 16, 16)  dtype: torch.float32  min:       -inf,  max:   19.97351,  mean:       -inf
h2            shape: (1, 64, 32, 32)  dtype: torch.float32  min:       -inf,  max:    9.99961,  mean:       -inf


In [18]:
valid_dl.dataset[0]

{'name': 'Z6MFQCViBuw_room0__0',
 'input': [tensor([[  1,  34,  43],
          [  1,  34,  45],
          [  1,  35,  44],
          ...,
          [127,  63,   7],
          [127,  63,   8],
          [127,  63,   9]]),
  tensor([[-2.9420],
          [-2.9810],
          [-2.8630],
          ...,
          [ 0.1011],
          [ 0.9870],
          [ 2.2126]])],
 'sdf': tensor([[[[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           ...,
           [   -inf,    -inf,    -inf,  ..., -3.6007, -3.5752, -3.5499],
           [   -inf,    -inf,    -inf,  ..., -3.6176, -3.5917, -3.5625],
           [   -inf,    -inf,    -inf,  ..., -3.6344, -3.6072, -3.5716]],
 
          [[   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -inf,    -inf,  ...,    -inf,    -inf,    -inf],
           [   -inf,    -

In [19]:
valid_ds = valid_dl.dataset
fn = valid_ds.files[0]
fn

'/home/goodok/p_ssd/sgnn/sgnn/data/completion_blocks/Z6MFQCViBuw_room0__0.sdfs'

In [21]:
valid_ds.is_chunks

True

In [22]:
Path(fn).with_suffix('').name

'Z6MFQCViBuw_room0__0'

In [23]:
item = data_util.load_train_file(fn)

In [24]:
item

([array([[  0,  20,  41],
         [  0,  21,  42],
         [  0,  22,  41],
         ...,
         [127,  63,   9],
         [127,  63,  10],
         [127,  63,  11]], dtype=int32),
  array([-4.945457 , -4.6241403, -4.7895193, ...,  2.2126198,  3.1741421,
          4.1063905], dtype=float32)],
 array([[[      -inf,       -inf,       -inf, ...,       -inf,
                -inf,       -inf],
         [      -inf,       -inf,       -inf, ...,       -inf,
                -inf,       -inf],
         [      -inf,       -inf,       -inf, ...,       -inf,
                -inf,       -inf],
         ...,
         [      -inf,       -inf,       -inf, ..., -3.600729 ,
          -3.5752122, -3.5498884],
         [      -inf,       -inf,       -inf, ..., -3.6175673,
          -3.5917196, -3.5624607],
         [      -inf,       -inf,       -inf, ..., -3.6344059,
          -3.607198 , -3.571591 ]],
 
        [[      -inf,       -inf,       -inf, ...,       -inf,
                -inf,       -inf],